# Introduction

One of HiSparc's goals is to bring real science to high school classrooms. This document is part of that. Here you will find walkthrough of Python code that does callibration for HiSparc detectors. Along with explanation of bits of the code, this notebook also provides excersises to help you get used to Python and real data analysis.

If you follow this entire notebook you will have learned the following:
* Why Python is used in modern science.
* What Python packages are and what they do.
* How to read data files using Python.
* What binning is and why it's useful.
* How to make a simple graph using Python and Pylab.
* How to read and interpret your graphs.
* How to use the code in this notebook to check if your school's detectors are working well.

A lot of functions of the code in this notebook are beyond the scope of these teaching goals. To still try to satisfy the curious reader some additional reading will also be given so that you can try and figure out how it works for yourself. Perhaps in the future this notebook will expanded to also explicitly cover the more complicated aspects of the code.

Before we get started, this notebook will assume you already know some things. These subjects are listed below. If you're note familiar with one or more of them make sure you read or ask a teacher about them before you continue. 
* You are familiar with high school level mathematics.
* You know what HiSparc is and what research they do.

The dirst two sections will focus on coding and programming skills, the third chapter will be a mix of programming and data analysis while the last two chapters focus solely on data analysis. Now that all that is out of the way let's begin.

# Using Python

Python is a programming language often used by scientist to make scientific models or to analyse data. Perhaps you've already made a graph using Microsoft Excel before or even some basic modelling using programs like CMA's Coach. These programs are great for small data sets or smaller simulations, however when doing new science we often work with Terabytes of data, if not more. When working with so much data excel will simply crash, let alone let you manipulate the data or make graphs. 

To test this let's try to open the "Excel Data.text" file in the same folder as this notebook with Excel.

This is slightly modified data file from HiSparc. You can use the link above to download it, it's about 200MB. Once you've done that open it with windows notepad (kladblok). It will open quite quickly, unfortunatly notepad has less graphing capability than an actual physical notepad. Don't close the notepad just yet. Let's open the file in excel as well first. To do this open excel and go to file -> open -> browse. Make sure to select "all files" (this can be found in the bottom right of the browsing window) in the browsing window. Now select the file you just opened. It will open a new window to let you import a .txt type file into excel.  Make sure to select "delimited" (this should be automatically selected for you) in this window and then press next. In the next window make sure to also select "other" and type "." (without the "", so just the period) in the text field next to other. Once you've done that, you can press finish. Excel will now freeze, this is normal after some time (you can try to count how long it takes if you wish) the file will open. 

It will probably give an error message telling you it could not load the entire file. We can check how much of the data it actually loaded by scrolling all the way down and checking the date (left most colomn). You can compare that to the latest date in notepad, they probably don't match. This is because excel just gives up trying to load more data after a while. Needless to say this is really bad if we want to actually use all the data.

Using python we can actually use the entire data set. So let's set that up here. This notebook has a very nice property, aside from displaying text it can also run python code like below. 

In [ ]:
print("Hello World!")

The bit of code above is usually the first bit of code anyone trying to learn programming will write. It simply writes the text "Hello World!" in the output line. You can run the code by clicking on it to select the block of code and then pressing ctrl + enter or by pressing the play (run) button on the top of this window. When you do so the text "Hello World" will appear below the code. That area is called the output line. It is where any text, images or error messages will get displayed by the code.

Before we can begin programming ourselves I highly recommend you look up an online Python tutorial to learn more. Here are some examples:
* https://youtu.be/rfscVS0vtbw (Free)
* https://www.w3schools.com/python/default.asp (Free)
* https://www.learnpython.org/ (Free)
* https://realpython.com/ (Paid)

The first thing we will want to do is import some packages. Python can do a lot on it's own already but not everything. For example making a graph isn't possible without first writing an entire complicated program that can make them for you. This is where packages come in. One of the wonderful things about python is that many others have already written programs for many of the things you might want to do and then made those programs available for the public in packages. These packages can be installed and imported so that you can use the code other have already written. In the code block below we will import some packages for our program that will allow us to do more complicated math and make our own graphs easily.

In [ ]:
#Import Packages
import time

import numpy as np

import pylab as pl

import tkinter.filedialog as tk

from scipy.optimize import curve_fit

from scipy.stats import chi2

Now that we have done that we can begin by importing data from HiSparc into our program in a way that we can use.

# Importing HiSparc Data

First you will need to download some real data from the HiSparc website here: https://data.hisparc.nl/data/download/.
Here you must select a station of your liking. We recommend 501: Nikhef or your own school if it has a detector with some recently collected data. For the date we recommend selecting a recent starting date and the day after it as an end date. You can select a larger timespan but this increases the size of your data file by a lot. This will make the download take longer as well as cause the program to take drastically longer to finish its computation. 

Next we need to get the information in the file you just downloaded into python. We'll be using the package "Tkinter" for this. Tkinter is a package that let's you import files into your code with a nice window like you're used to. First we need to set up tkinter. We do this by creating a "root" window. This is an empty window in which we can display things we want. But we don't want to display an empty window so we'll make it invisible right away as well. Then we can open a file selection window on top of the empty window. This window will then import the file(s) we selected into python, we will call them "files" to refer to later in the code. Last we need to close the window. To do all this we need the following code. Note that everything after the "#" simple doesn't add to the code itself but is instead a note describing what a line or some lines do.

If no window opens when you run the cell below, use "alt+tab" to cycle through all open windows. You will most likely find it there.

In [ ]:
root = tk.Tk() #Create root window
root.withdraw() #Make the root window invisible
files = tk.askopenfilenames(parent=root, title='Choose any files') #Open a file explorer and import select files as "files"
root.destroy() #Close all windows

If this seems like a lot at once, that's okay because it is. You don't have to be able to do this yourself. The important take away is that we now have our file(s) imported into python and called them "files".  Notice that we could select more than one file. This is because "files" is actually list in which we can put multiple files. We need to keep that in mind in the future. 

We're not done yet however. Currently we have list with our files in it, however we want python to read what is in those files. Try to open the file you just downloaded with notepad to see what it looks like. It should look something like this:
![title](file_example.png)

First are a bunch of lines describing what is in the file. The third of those is the name of the station the data was taken from, the fifth the start and end date of the data in the file. Then follow some lines about the license for using the data. Lastly are some lines telling you what information is found in what column below.

After that we can see columns of numbers, which is the actual data. A nice way for python to read this would be as follows: 

-Make a list for each column of data and put those lists into a list of lists.

-Make the first "element" of that list the name of station so we can identify it later.

-Go through each line in and ignore it if its the text we don't care about.

-For each line with data, add the number in each column into the list that corresponds to that column.

When we do this for each line in the file we should end up with a list that contains a list for each column of data as well as the name of the station. This means that the first list in our list of lists will have the name of the station, the second list will have the date, the thirs list will have the time of the measurement etc. etc. You can read the file to check what each column is.

Below is the code that will do all this. Try to follow along with every step, if you don't understand what a bit of code does, you can always google it (for example you could google "Python append()" to figure out what append means).
When beginning to learn how to code don't be afraid to google a lot of things, even experienced programmers will continue to do this. It is a really good way to learn how to code. A lot of people will have had the same ideas and problems as you and other will have figured out how to do it, so you don't have to reinvent the wheel.

Keep in mind that in python we count starting form 0, not 1. So the first column is actually column number 0 while the second column is column number 1.

In [ ]:
stations = [] #Make a list in which we can store the data of each file in a useful way, call that list stations

for file in files: #Loop through every file in the list files using the for loop, read as: "For each file in files do the following:"
    
    with open(file) as fa: #Open the file and call it "fa"

    #Read all lines
        station = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []] #Make a list of lists with a list for each column + 1 for the name of the station. We call this list of lists "station".
        for line_aa in fa.readlines(): #For every line (which we will call line_aa) in the file do the following

            if line_aa.startswith("# Station:"): #If the line starts with "# Station:" the do the following
                a = line_aa.split(':',2) #Split the line in half after the : and put both halves in a list called a, the second half will be the name of the station
                station[0].append(a[1]) #Add the second half the line (the name of the station) to the first list in station
            elif len(line_aa.split('\t',23))==23: #If the line doesn't start with "# Station:" then check if you can split it into 23 columns, if you can then do following
                a = line_aa.strip() #Remove all extra spaces, tabs and symbols
                cols = line_aa.split('\t',23) #Split the line at every tab. This will split all columns and put each number in a column in a list called cols

        #Now you have all the colums. Col1 through to Col4 countain data about the time of events. Col5 through to Col8 contain pulseheights of events. Col9 			through to 12 contain pulse integrals. Col 13 through to 16 contain the estimate for the number of particles detected. Col17 through to 20 contain the relative time between detections. Col 			21 contains trigger times. Col 22 and 23 the zenith and azimuth of the shower (currently not useable).
                for i, col in enumerate(cols): #For each column in the list col do the following, keep track of which column we're at with the variable i
                    if(i>3): #If it is the 5th column or above then do the following
                        station[i+1].append(float(col)) #Add the content of the column to the list of lists as a "float" (a number). Add it to the i+1th list in station (not the ith because the 1st list already has the name of station in it so we wanna skip that one) 
                    else: #If it's one of the first 4 columns then do the following
                        station[i+1].append(col) #Add the content of the column as a bit of text (a "string"). Add it to the i+1th list in station
            else: #If you the line doesn't start with "# Station:" and can't be split into 23 columns then ignore it
                continue
        stations.append(station.copy()) #Add a copy of the list of lists to list of stations (We're now three lists deep)
        station.clear() #Clear the contents of station so we can use it again for the next file


Okay that was a lot to go through, if you could follow all that, great job! If not, that's perfectly understandable. Try to go through it a few more times. If still don't understand what's going on try to ask a fellow student that does or perhaps a teacher. In the end the most important part to get is that we now have a list called stations. In that list we have a list, or multiple lists if we import mulitple files. This list (or lists) has the data of one station in it, let's therefor refer to each list in stations as a "station" for now. In each station are a number of lists (yes that's a lot of lists) the first list (list number 0 in python) contains the name of the station, every list after that contains data from each column in the file.

The next step is doing something useful with the data. Like making a graph of it!
# Making a Graph 

There are many different graphs that you can make in Python. However Python itself cannot make any graphs, the "Pylab" package is needed. If you look back at the second cell of code we wrote, [2], we imported pylab as "pl". This means that whenever we want to use a function in the pylab package we need to start by typing pl. followed by the function, so: 'pl.function()'. If you wanna learn more about what the pylab package can do check out this link: https://matplotlib.org/tutorials/introductory/pyplot.html Keep in mind that tutorials like the one in that link are written for people who know Python well, so don't expect to immediately understand everything. The part of pylab that we're going to be using is called pyplot.

Let's start by recreating a graph made by HiSPARC. If you go to https://data.hisparc.nl/show/stations_by_country/ and click on the name of a station (that of your school should be on here) that is active (one with a green dot and square in front of the name), you will see a few graphs. We are going to recreate the "Pulseheight Histogram". Which should look something like below. This graph shows us how many particles were detected (count) with a certain energy which the detector picks up as a pulse with a certain height (so pulseheight).

![title](pulseheight_hist.png)

To do that we will have to make a histogram. The pyplot function that does this is called hist (https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html#matplotlib.pyplot.hist). Using it will actually be very easy. All we need to do is write what data we want to make a histogram of inside the brackets of the hist() function. The data we wanna use here is the pulseheight of one of the detectors of the data we imported. Let's use the first one. That would be called stations[0][5], because we want to use the first (and only) station in stations (remeber that python starts couting at 0, not 1) and the fifth column in that station.


#### Exercise 1.1


Try to write the code for the histogram below and run the cell:

Even if you did it right no graph will have appeared on screen. This is because we will also have to tell the code to actually show the graph with the show() function.

#### Exercise 1.2


Update your code show that it shows your graph.

If you did it right you should now have something that looks like this:

![title](First_attempt.png)


Okay that is still nothing like what we want. There's still quite a few differences between our graph and that of HiSPARC. First our bars are much broader than they should be. We also have bars instead of lines. Also if you look closely the scale on the y-axis is very different. HiSPARC's plot also has four lines and some nice axes labels. Lastly HiSPARC has 4 lines, not 1. All of these we are going to solve step by step.

First the width of our bars. These bars are usually called bins. Bins are also used for different graphs. It is almost never useful to show the exact data you collected. For example here we are trying to show how many particles with a certain pulse height were detected. However if we showed how many particles were detected for every value between 0 and 4000 seperatly we would have 4000 bars in one graph and almost all of them would be very low. It would be super hard to read such a graph! That's not all however because doing calculations with all our raw data can take a very long time if we have to do them for every single particle we detected. Bins are the solution to both these problems. What we do instead is group many particles together into a single bin. So instead of showing how many particles there were with a pulse heigth of 328, we show how many there were with a pulse height between 300 and 400! This way we only have 400 bars to show and each one will have many particles in it. Further more if we want do calculations with this then we only have do to 400 of them isntead of 4000! For this reason bins are super useful. For the rest of the excersis we will be dividing up the data in 250 bins.


#### Exercise 1.3

Use the bins below to update your graph with 250 bins, each 16.8 mV wide.

*Hint:* You will have to update what's inside the hist() function, when doing this always seperate different "arguments" (that's what we call whatever is inside the brackets of a function) with a comma. Check the tutorial and the hist documentation for examples!


In [ ]:
bins = np.linspace(0, 4200, 250)

If you did it right you should now have something that looks like this:

![title](bins.png)

Which is still not much better, maybe even worse! Okay maybe it is time to fix those axes. The problem is that the HiSPARC graph uses a logarithmic scale (log scale for short), so we should probably do that too


#### Exercise 1.4


Using https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html#matplotlib.pyplot.hist find a way to get a log scale on your graph and update your graph using that scale.

*Hint:* You'll need to update the hist() function again, don't forget a comma!

You should now have a a graph that looks like this now:

![title](log_scale.png)

Notice that the scale on the y-axis is now in powers of 10, here's the wikipedia article for the log scale if you want to learn more: https://en.wikipedia.org/wiki/Logarithmic_scale if you want to learn more.

Next to make it more readable we want to turn the solid bars into lines like in the graph made by HiSPARC, this will be done in a very similar manner as the previous step. This step is mostly useful later when we are adding more than one detector in the same graph, as the solid bars would be in front of each other.


#### Exercise 1.5

Again using https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html#matplotlib.pyplot.hist find a way to have the graph be made out of lines instead of solid bars.

*Hint:* You're looking for the histogram type, histtype for short.

You should now have something like this:

![title](lines.png)

This is already much better than what we started with. After all we want to be able to get information out of the graph and that wasn't at all possible with our first attempt.

Next we will be adding labels to our axis to show what they represent. We need to use the xlabel() and the ylabel() functions. These need to come after the histogram but before the show() function. https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xlabel.html has a collection of examples using labels.

#### Exercise 1.6

Add labels to your graph using the function mentioned below. Use the same text in the labels as the HiSPARC plot does.

*Hint:* Labels need to be text, or a "string" as it is called. To turn anything you write into a string it must be between apostrophes 'like so'.

You should now have this:

![title](labels.png)


The last step is to add the data from all 4 detectors to the graph like in the original. To do this you simply copy the hist() function you already made and only change what column you're showing on your graph. 

#### Exercise 1.7

Add the other three detectors to the graphs.

*Hint:* The other detecors are in column 6, 7 and 8.

And we're done, this is what the end result should look like:

![title](final.png)

It isn't a 1:1 match but it's really close. If you want more practice try using all  the information on https://matplotlib.org/tutorials/introductory/pyplot.html to get it as close as possible to the original.
# Analyzing the Graph

Graphs are made for a reason, to get information out of them. Here we will analyze the graph you made and compare it to another similar graph.

An important part of analyzing a graph is describing its shape and features. Specifically we're looking for a few things: What are the domain and range of the graph, where does the graph have extreme values (maximum or minimum values), where is the graph increasing and decreasing and is it doing so quickly or slowly and lastly are there any points where the graph makes any sudden jumps up or down (where is the graph *discontinuous*)? 

#### Exercise 2.1

Answer all the above questions for the graph you made. Make sure to report the pulse height of all the graphs features.

#### Exercise 2.2

Make the same graph you made before but now for columns 9, 10, 11 and 12. These columns contain the pulse integral (related to the width of the pulse) of all events.

*Hint:* Use the bins below:

In [ ]:
bins = np.linspace(0, 100000, 250)

It should look something like this:

![title](pulseint_hist.png)

#### Exercise 2.3

How are the two graphs you made different? How are they the same?

*Hint:* Do exercise 2.1 for the new graph you made and compare the differences

A minor difference between the two graphs is hard to spot yet very important. Once the second graph you made hits 1 for the first time it continous on for longer around 1 than the first graphs does. This makes the second peak look a lot more narrow.
This is because the sensor in the detector used by HiSPARC has a maximum value it can detect for the pulse height, but not the pulse integral. As a result a part of the data has to be disregarded before you can any real analysis with it. The value for the pulse height after which the data has to be discarded can be calculated. This value is called the saturation point of the detector and is different from detector to detector. Note that the saturation point is actually lower than the maximum value for the pulse height. Calculating the saturation point is a complicated process that we won't go into here. The saturation point can used to check if a detector should be replaced or not. In the next section we're going to check if the detectors at your school are any good.

# Comparing Stations

The saturation point can be anywhere from around 500 mV to 4000 mV. Anything below 1000 mV is definitly too low. Above 1500 mV is okay and above 2500 mV is really good. To figure out what the saturation points of the detectors at your school are you have to have data from your school imported into this document. If you did you can run the code below to get a plot that shows the saturation point of all detectors at your school. Running this code can take a little while! 

If the code takes up too much code screen space you can minimize it after running it by clicking on the blue to the left of the code.

In [ ]:
#Function used to fit the saturation plot

def linfit (x, c1):
	
    return c1*np.float64(x)

def linfit2 (x, c1, c2):
	
    return c1*np.float64(x)+c2

def tanhfit (x, c1, c2):
    
    return -0.5*np.tanh(c1*np.float64(x)+c2)+0.5

#General Functions
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    val = array[idx]
    return idx, val

def chisq (ph, phfit, nprm=1,sd=None):
	"""Gives either the reduced chi squared of phfit with respect to ph if sd is given or normal chi squared if sd is none."""

	if sd==None:
		chisq = np.sum(np.power(ph-phfit,2))  
	else:
		chisq = np.sum(np.power(ph-phfit, 2)/np.power(sd,2))  
	return chisq 

def pvalue (rcsq, deg):
    """Returns p-value when given a reduced chi square statistic and degrees of freedom"""    		

    pv = chi2.sf(rcsq, deg)

    return pv
    
#Declare Classes

class saturation:
    """A class used for finding the satuartion point of a station."""
    #Initialization of Class	
    def __init__(self, station):
            self.phs = [station[5], station[6], station[7], station[8]]
            self.pis = [station[9], station[10], station[11], station[12]]
            self.name = station[0]

	#given a set of bin edges returns the average pulse height and the average pulse integral of pulse heights between given edges. Empty bins are discarded. 
	#Also returns the standard deviations in both the pulse intregrals and pulse heights
    def avgs(self, bins = (np.linspace(0,150000, num=120)).tolist()):
        pihs = []
        for i, ph in enumerate(self.phs):
            pih = list(map(lambda x, y:(x,y), self.pis[i], ph))
            pih.sort(key=lambda tup: tup[0])
            pihs.append(pih.copy())
        
        avgphs = []
        stdphs = []
        avgpis = []
        stdpis = []
        
        for pih in pihs:
            temp = [[],[]]
            avgph = []
            stdph = []
            avgpi = []
            stdpi = []
            counter = 1
            for pi in pih:
                while counter<len(bins):
                    if pi[0] < bins[counter]:					
                        temp[0].append(pi[0])					
                        temp[1].append(pi[1])
                        break
                    elif temp[0] and temp[1]:
                        counter += 1
                        """avgpi.append(np.average(temp[0]))
                        stdpi.append(np.std(temp[0]))
                        avgph.append(np.average(temp[1]))
                        stdph.append(np.sqrt(np.sum(np.power(np.sqrt(temp[1]),2))))
                        temp = [[],[]]
                        continue"""
                        if len(temp[0]) > 5:
                            avgpi.append(np.average(temp[0]))
                            stdpi.append(np.std(temp[0]))
                            avgph.append(np.average(temp[1]))
                            stdph.append(np.std(temp[1]))
                            temp = [[],[]]
                            continue
                        else:
                            avgpi.append(temp[0][0])
                            stdpi.append(1)
                            avgph.append(temp[1][0])
                            stdph.append(np.sqrt(np.average(temp[1])))
                            temp = [[],[]]
                            continue				
                    else:
                        counter += 1
                        continue
            avgphs.append(avgph)
            avgph = []
            stdphs.append(stdph)
            stdph = []
            avgpis.append(avgpi)
            avgpi = []
            stdpis.append(stdpi)
            stdpi = []
        return avgpis, stdpis, avgphs, stdphs

	#Gives parameters for fit using func. Takes the function used for the fit, a list of pulse intregrals, a list of pulse heights and a list of uncertainties in the pulse heights.
	#Returns the parameters of the fit and the chi square with p-value of the fit.
    def giveFitParam(self, func, x, y, yerr):	
        paramc = curve_fit(func, x, y, sigma=yerr, method='lm')
        param = paramc[0]
        csq = chisq(y, func(x, *param), sd=yerr)
        return param, csq
	
    def findChiSqrs(self, bins = (np.linspace(0,150000, num=120)).tolist(), startPoint = 2):
        pis, spis, phs, sphs = self.avgs(bins = bins)
        cpis = []
        cphs = []
        csqs = []
        pvs = []
        prms = []
        for j, ph in enumerate(phs):
            i = 4
            i += 1
            cpi = []
            cph = []
            csq = []
            pv = []
            prm = []
            if(ph and pis[j]):
                while i <= len(ph):
                    parm, c = self.giveFitParam(linfit, pis[j][startPoint:(startPoint + i)], ph[startPoint:(startPoint + i)], sphs[j][startPoint:(startPoint + i)])
                    cpi.append(pis[j][i-1])
                    cph.append(ph[i-1])
                    csq.append(c)
                    pv.append(pvalue(c, len(ph[startPoint:(startPoint + i)])-1))
                    prm.append(parm)
                    i += 1
                    
            cpis.append(cpi)
            cpi = []
            cphs.append(cph)
            cph = []
            csqs.append(csq)
            csq = []
            pvs.append(pv)
            pv = []
            prms.append(prm)
            prm = []
        return cpis, cphs, csqs, pvs, prms
    
    def findSatPoint(self, startpoint = 0):
        """Finds saturation point for this station station"""
        cpis, cphs, csqs, pvs, prms = self.findChiSqrs()
        
        prms = []
        npvs = []
        perrs = []
        satpts = []
        saterrs = []
        x =  np.linspace(0, 4500, 10000)
        for i, pv in enumerate(pvs):
            if pv:
                #Make fit to pvalue data and calculate pvalue of fit
                param, cov = curve_fit(tanhfit, cphs[i][startpoint:], pv[startpoint:], p0=[0.005, -10], sigma=0.01*np.ones(len(pv[startpoint:])))
                perr = np.sqrt(np.diag(cov))
                perrs.append(perr)
                prms.append(param)
                ncsq = chisq(pv[startpoint:], tanhfit(cphs[i][startpoint:], *param))
                npv = pvalue(ncsq, len(pv)-2)
                npvs.append(npv)
                
                #Find saturation point
                idx, val =  find_nearest(tanhfit(x, *param), 0.9)
                satpt = x[idx]
                satpts.append(satpt)
                
                #Find saturation point error
                param_plus = param + perr
                param_min = param - perr
                idx_plus, val_plus =  find_nearest(tanhfit(x, *param_plus), 0.9)
                saterr_plus = np.abs(x[idx] - x[idx_plus])
                idx_min, val_min =  find_nearest(tanhfit(x, *param_min), 0.9)
                saterr_min = np.abs(x[idx] - x[idx_min])
                saterr = [saterr_min, saterr_plus]
                saterrs.append(saterr)
                
                #Correction using heaviside step f
                if satpt > 3000:
                    
                    #Make fit to pvalue data and calculate pvalue of fit
                    param, cov = curve_fit(tanhfit, cphs[i][startpoint:], pv[startpoint:], p0=[0.005, -17.5])
                    perr = np.sqrt(np.diag(cov))
                    perrs[i] = perr
                    prms[i] = param
                    ncsq = chisq(pv[startpoint:], tanhfit(cphs[i][startpoint:], *param))
                    npv = pvalue(ncsq, len(pv)-2)
                    npvs[i] = npv
                
                    #Find saturation point
                    idx, val =  find_nearest(tanhfit(x, *param), 0.9)
                    satpt = x[idx]
                    satpts[i] = satpt
                
                    #Find saturation point error
                    param_plus = param + perr
                    param_min = param - perr
                    idx_plus, val_plus =  find_nearest(tanhfit(x, *param_plus), 0.9)
                    saterr_plus = np.abs(x[idx] - x[idx_plus])
                    idx_min, val_min =  find_nearest(tanhfit(x, *param_min), 0.9)
                    saterr_min = np.abs(x[idx] - x[idx_min])
                    saterr = [saterr_plus, saterr_min]
                    saterrs[i] = saterr
                """   
                #Make plot of p-values and fit
                pl.subplot(2,2,i+1)
                pl.plot(x, tanhfit(x, *param), label='Fit of P-values')
                pl.plot(x, tanhfit(x, *param_min), '--k', label='Fit with Error')
                pl.plot(x, tanhfit(x, *param_plus), '--k')
                pl.plot(cphs[i], pv, '.', label='P-values')
                pl.xlabel('Pulse Height (mV)')
                pl.ylabel('P-Value')
                pl.hlines(0.9, 0, 4500, 'r', label='Threshold')
                pl.xlim(0,4500)
                pl.ylim(-.1,1.1)
                pl.title('Fit of P-values of detector' + str(i+1))
                pl.legend()
                
        pl.show()
        """            
        return satpts, saterrs, npvs, prms
    
#Create comparison plot of all stations, assuming data given was from different stations.
  
#First create a list of all instances of the saturation class
classes = []
stations.sort(key=lambda x: x[1][0])
for station in stations:
    classes.append(saturation(station))

#Create list of saturation points and their errors 
satpts = []
saterrs = [[],[]]
for cl in classes:
    satpt, saterr, pv, prms = cl.findSatPoint()
    for i, spt in enumerate(satpt):
        satpts.append(spt)
        saterrs[0].append(saterr[i][0])
        saterrs[1].append(saterr[i][1])
     
#Create list of station names
names = []
for i, satpt in enumerate(satpts):
    names.append(stations[i//4][0][0] + str((i % 4)+1))

#Create actual plot
x = np.arange(len(satpts))
pl.errorbar(x, satpts, yerr=saterrs, fmt='none', capsize=2)
pl.plot(x, satpts, '.r')
pl.xticks(x, names, rotation='vertical')
pl.xlabel('Station Name and Detector Nr.')
pl.ylabel('Saturation Point (mV)')
pl.ylim(0, 4500)
pl.show()

Depending on what is on the top of your school's roof you are now looking at either 2 or 4 dots with error bars. The error bars might be incredibly large, if that's the case you can ignore them, it's okay. You already have some idea of what should be good values but lets compare your results to those of the Nikhef cluster. This is a large group of HiSPARC stations in Amsterdam Science Park. Below you will find the same plot as your but for all the stations in that cluster. 

![title](Figure_8.png)

#### Exercise 3.1

Compare the saturation points of the detectors at your school to those at the Nikhef cluster. Are your detectors below average, average or above average? 

#### Exercise 3.2

Try to identify a station where there might be something wrong with at least 1 detector. Do you think only one or all detector should be replaced or do you need more information?

If you came to the conclusion that there's detector(s) at your school that aren't working properly it might be time to replace them. However it could be that the detectors is actually working fine but there objects in its environment that are disrupting it. To check if that's here's some things you could check: Are there any metal object on your school's roof close or directly over the detector? Are there any strong antennas nearby? Is there work being done with powerful (like construction site machines) machinery near your school? Is the detector installed correctly? 

There are many reasons why a detector might have a low saturation point, and it being faulty is only on of them. Trying to find what's wrong could be whole project on its own!

# Closing Words

This is the end of this document. We hope you managed to learn something. Be sure to check the answers to the exercises which should be in the same folder as this document!

For now work on this document has stopped, however more people could pick it back up again in the future. We would like to have your feedback so this document can be improved in the future. If you have any idea to improve this document please shar them with your teacher so they can pass your feedback back to us.